In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

llm = ChatOpenAI()

memory = ConversationBufferMemory(return_messages=True, return_message=True,)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./rag_data/chapter_3.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

prompt_template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. {context}

Question: {question}
Answer: 
"""

prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
)

chain = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff", 
	retriever=vectorstore.as_retriever(),
	memory=memory,
	chain_type_kwargs={"prompt":prompt},
)

def invoke_chain(question):
	result = chain.invoke({"query":question})
	print(f"Question: {question}")
	print(f"Answer: {result['result']}")

In [20]:
invoke_chain("Aaronson 은 유죄인가요?")

Question: Aaronson 은 유죄인가요?
Answer: I don't know.


In [23]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

Question: 그가 테이블에 어떤 메시지를 썼나요?
Answer: 그가 쓴 메시지는 "FREEDOM IS SLAVERY"이었고, 그 아래에 "TWO AND TWO MAKE FIVE"와 "GOD IS POWER"라고 썼습니다.


In [22]:
invoke_chain("Julia 는 누구인가요?")

Question: Julia 는 누구인가요?
Answer: Julia is a character in the novel who is involved in a romantic relationship with Winston, the main character.


# RAG(Retrieval-Augmented Generation) 시스템에서의 Context

## 질문
"주어진 코드에서 `context` 변수가 불필요한 것 아닌가요? 명시적으로 정의되지 않았는데 왜 프롬프트 템플릿에 그것의 자리 표시자가 있나요?"

## 답변
`context` 변수는 코드에서 명시적으로 정의되지 않았지만, RAG 시스템의 중요한 구성 요소입니다. 자세한 설명은 다음과 같습니다:

### 1. 프롬프트 템플릿에서 `{context}`의 역할

프롬프트 템플릿에는 `{context}`를 위한 자리 표시자가 포함되어 있습니다:

```python
prompt_template = """
다음의 컨텍스트를 사용하여 마지막에 있는 질문에 답하세요. 
답을 모르는 경우, 모른다고 말하고 답을 만들어내려 하지 마세요. {context}

질문: {question}
답변: 
"""
```

이 자리 표시자가 필요한 이유는:

- 시스템이 검색한 관련 문서 조각들로 채워질 것입니다.
- 언어 모델(LLM)에게 질문에 정확히 답하기 위해 필요한 배경 정보를 제공합니다.

### 2. `context`가 처리되는 방식

명시적으로 정의되지 않았지만, `context`는 RAG 시스템 내부에서 다음과 같이 관리됩니다:

1. `vectorstore.as_retriever()`가 사용자의 질문을 기반으로 관련 문서 조각들을 검색합니다.
2. 이 조각들이 `context`로 사용됩니다.
3. RetrievalQA 체인이 자동으로 이 `context`를 프롬프트 템플릿에 주입합니다.

### 3. `{context}` 자리 표시자의 중요성

`{context}` 자리 표시자를 제거하면 안 되는 이유는:

- 각 쿼리에 대해 관련 정보를 주입하는 곳이기 때문입니다.
- 제거하면 RAG 시스템의 핵심 기능이 비활성화됩니다.
- 이것이 없으면 LLM은 사전 학습된 지식에만 의존하게 되어, 덜 정확하거나 관련성이 낮은 답변을 제공할 수 있습니다.

### 4. 동적 컨텍스트 주입

- `{context}` 자리 표시자를 채우는 내용은 각 쿼리마다 변경됩니다.
- 이 동적 주입을 통해 시스템은 각 특정 질문에 가장 관련된 정보를 제공할 수 있습니다.

### 5. 프롬프트 엔지니어링의 일부

- `{context}`의 배치와 사용은 프롬프트 엔지니어링의 중요한 측면입니다.
- 이는 LLM에게 제공된 정보를 바탕으로 답변하라고 효과적으로 지시합니다.

## 결론

프롬프트 템플릿의 `{context}` 자리 표시자는 RAG 시스템의 핵심 구성 요소입니다. 이를 통해 관련 정보를 동적으로 주입할 수 있어, LLM이 검색된 문서 조각들을 바탕으로 정확하고 컨텍스트에 특화된 답변을 제공할 수 있게 됩니다. 사용자의 코드에서 명시적으로 정의되지 않았지만, RetrievalQA 체인 내부에서 처리되어 시스템 기능의 핵심 부분을 형성합니다.